In [1]:
import requests
import pandas as pd
import ast
import re
import json

In [2]:
XAI_API_URL = "https://api.x.ai/v1/chat/completions"

In [3]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [4]:
def extract_valid_list(response_text):
    
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text)  
    response_text = response_text.strip("`")  

    match = re.search(r"\[[^\]]+\]", response_text, re.DOTALL)  

    if match:
        list_text = match.group(0)
        try:
            extracted_list = ast.literal_eval(list_text)  
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception as e:
            print(f"Error parsing response: {e}")  
            return []  
    
    return []

In [5]:
def generate_ner_tags(sentence, api_key):

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "grok-2-latest",
        "messages": [
            {"role": "user", "content": f"""
            You are a business process modelling expert, tasked with identifying which mentions of certain process relevant elements in textual descriptions of business processes belong to the same entity. 

            Allowed NER tags and their explanations:
            - O: An O tag indicates that a token belongs to no chunk.
            - B-Actor: This tag indicates the beginning of an Actor chunk.
            - I-Actor: This tag indicates that the token is inside an Actor chunk.
            - B-Activity: This tag indicates the beginning of an Activity chunk.
            - I-Activity: This tag indicates that the token is inside an Activity chunk.
            - B-Activity Data: This tag indicates the beginning of an Activity Data chunk.
            - I-Activity Data: This tag indicates that the token is inside an Activity Data chunk.
            - B-Further Specification: This tag indicates the beginning of a Further Specification chunk.
            - I-Further Specification: This tag indicates that the token is inside a Further Specification chunk.
            - B-XOR Gateway: This tag indicates the beginning of a XOR Gateway chunk.
            - I-XOR Gateway: This tag indicates that the token is inside a XOR Gateway chunk.
            - B-Condition Specification: This tag indicates the beginning of a Condition Specification chunk.
            - I-Condition Specification: This tag indicates that the token is inside a Condition Specification chunk.
            - B-AND Gateway: This tag indicates the beginning of an AND Gateway chunk.
            - I-AND Gateway: This tag indicates that the token is inside an AND Gateway chunk.
        
            An example of this could look like this:
            Sentence:
            ['The', 'post', 'is', 'advertised', ',', 'applicants', 'apply', ',', 'the', 'applications', 'are', 'checked', 'and', 'the', 'post', 'is', 'filled', '.']
            NER_tags:
            ['B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Actor', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O']
            
            
            Now, label each token in the following sentence with one of the above NER tags:
            
            Sentence: {sentence}
            
            Output ONLY a Python-style list of NER tags. Do not include any explanations or additional text.
            """}
                ],
        "temperature": 0.0,
        "max_tokens": 1024
    }

    try:
        response = requests.post(XAI_API_URL, headers=headers, data=json.dumps(payload))
        response_data = response.json()

        if "choices" in response_data and response_data["choices"]:
            response_text = response_data["choices"][0]["message"]["content"].strip()
            print(f"DEBUG: Raw API Response: {response_text}") 
            return extract_valid_list(response_text)
        else:
            print("DEBUG: Empty response from Grok API")
            return []
    
    except Exception as e:
        print(f"Error calling Grok API: {e}")
        return []

In [6]:
def process_dataset(input_file, output_file, api_key_file):

    api_key = get_api_key(api_key_file)
    dataset = pd.read_csv(input_file)

    def process_row(row):
        try:
            tokens = ast.literal_eval(row["tokens"])  
            sentence = " ".join(tokens)
            return generate_ner_tags(sentence, api_key)
        except Exception as e:
            return f"Error: {str(e)}"

    dataset["grok-2-latest_ner_tags"] = dataset.apply(process_row, axis=1)

    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [7]:
input_file = "PET-Customized.csv" 
output_file = "PET-Customized-Grok2Vision.csv" 
api_key_file = "./API_Key/xAI.txt"

if __name__ == "__main__":
    process_dataset(input_file, output_file, api_key_file)

DEBUG: Raw API Response: ['B-Actor', 'I-Actor', 'I-Actor', 'B-Activity', 'B-Activity Data', 'I-Activity Data']
DEBUG: Raw API Response: ['B-Condition Specification', 'I-Condition Specification', 'I-Condition Specification', 'B-Actor', 'I-Actor', 'B-Activity', 'I-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O']
DEBUG: Raw API Response: ['B-Actor', 'I-Actor', 'I-Actor', 'I-Actor', 'I-Actor', 'O', 'O', 'B-Actor', 'I-Actor', 'O', 'B-XOR Gateway', 'O', 'B-Activity', 'O', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'I-Activity Data', 'I-Activity Data', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'B-Actor', 'I-Actor', 'O', 'B-Actor', 'I-Actor', 'I-Actor', 'O', 'B-Activity', 'O']
DEBUG: Raw API Response: ['B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Acti